## N-queens with Ariel
This file demonstrates how to tackle the n-queens problem using Ariel. Solving for a board with n=8, n=16 and n=32 where n is the the board length and width and also the number of queens on the board. The goal is to achieve an average of zero attacking queens on all tested n values. This will demonstrate the power and usability of the Ariel library.

In [1]:
# Standard library
import random
from collections.abc import Callable
from dataclasses import dataclass
from pathlib import Path
from typing import Literal, cast

# Pretty little errors and progress bars
from rich.console import Console
from rich.progress import track
from rich.traceback import install

# Third-party libraries
import numpy as np
from pydantic_settings import BaseSettings
from sqlalchemy import create_engine
from sqlmodel import Session, SQLModel, col, select 

# Local libraries
from ariel.ec.a000 import IntegerMutator, IntegersGenerator
from ariel.ec.a001 import Individual
from ariel.ec.a005 import Crossover
from ariel.ec.a004 import EASettings, EAStep, EA

# Library to show fitness landscape
import matplotlib.pyplot as plt

In [28]:
type Population = list[Individual]
config = EASettings()
config.is_maximisation = False  # We want to minimize the number of attacking queens
config.target_population_size = 100
config.num_of_generations = 100

In [10]:
def visualize_solution(solution):
    """Visualize the placement of queens on the chessboard."""
    n = len(solution)
    for i in range(n):
        e = '. '
        row_str = e * (solution[i]) + 'Q ' + e * ((n-1) - solution[i])
        print(row_str)
        
def count_attacking_queens(solution: list[int]) -> int:
    """Counts the number of attacking queens on diagonals."""
    fitness = 0
    main_diags, anti_diags = [], []
    
    for row, col in enumerate(solution):
       if row - col in main_diags:
          fitness += main_diags.count(row - col)
       if row + col in anti_diags:
          fitness += anti_diags.count(row + col)
      
       main_diags.append(row - col)
       anti_diags.append(row + col)       

    return fitness


In [47]:
def parent_selection(population: Population, k=2) -> Population:
    """Tournament selection of size k"""
    random.shuffle(population)
    for idx in range(1, len(population) - 1, k):
        ind_i = population[idx]
        ind_j = population[idx - 1]

        # Compare fitness values
        if ind_i.fitness < ind_j.fitness and not config.is_maximisation:
            ind_i.tags = {"ps": True}
            ind_j.tags = {"ps": False}
        else:
            ind_i.tags = {"ps": False}
            ind_j.tags = {"ps": True}
    return population


def crossover(population: Population) -> Population:
    """One-point crossover"""
    parents = [ind for ind in population if ind.tags.get("ps", False)]
    for idx in range(1, len(parents), 2):
        parent_i = parents[idx - 1]
        parent_j = parents[idx]
        genotype_i, genotype_j = Crossover.order_cross(
            cast("list[int]", parent_i.genotype),
            cast("list[int]", parent_j.genotype),
        )

        # First child
        child_i = Individual()
        child_i.genotype = genotype_i
        child_i.tags = {"mut": True}
        child_i.requires_eval = True

        # Second child
        child_j = Individual()
        child_j.genotype = genotype_j
        child_j.tags = {"mut": True}
        child_j.requires_eval = True

        population.extend([child_i, child_j])
    return population


def mutation(population: Population) -> Population:
    for ind in population:
        if ind.tags.get("mut", False):
            genes = cast("list[int]", ind.genotype)
            mutated = IntegerMutator.integer_swap(
                individual=genes,
                mutation_probability=0.5,
                swaps=1,
            )
            ind.genotype = mutated
    return population

def evaluate(population: Population) -> Population:
    for ind in population:
        ind.fitness = count_attacking_queens(ind.genotype) 
        ind.requires_eval = False
            
    return population

def survivor_selection(population: Population) -> Population:
    """Tournament selection of size 2"""
    random.shuffle(population)
    current_pop_size = len(population)
    for idx in range(1, len(population)):
        ind_i = population[idx - 1]
        ind_j = population[idx]

        # Kill worse individual
        if ind_i.fitness < ind_j.fitness and not config.is_maximisation:
            ind_j.alive = False
        else:
            ind_i.alive = False

        # Termination condition
        current_pop_size -= 1
        if current_pop_size <= config.target_population_size:
            break
    return population


def create_individual(n) -> Individual:
    ind = Individual()
    ind.genotype = IntegersGenerator.integers(low=0, high=n-1, size=n, replace=False)
    ind.tags = {"ps": False, "mut": False} # Initially not selected for anything
    return ind


def learning(population: Population) -> Population:
    return population

In [ ]:

def solve_problem(n) -> None:
    # Create initial population
    population_list = [create_individual(n) for _ in range(config.target_population_size)]
    population_list = evaluate(population_list)

    # Create EA steps
    ops = [
        EAStep("evaluation", evaluate),
        EAStep("parent_selection", parent_selection),
        EAStep("crossover", crossover),
        EAStep("mutation", mutation),
        EAStep("evaluation", evaluate),
        EAStep("survivor_selection", survivor_selection),
        EAStep("learning", learning),
    ]

    # Initialize EA
    ea = EA(
        population_list,
        operations=ops,
        num_of_generations=config.num_of_generations,
    )

    ea.run()

    return ea.get_solution()

In [48]:

solution = solve_problem(8)
print("Best solution found:")
print(solution.genotype)
print(visualize_solution(solution.genotype))
print(f"With { int(solution.fitness) } attacking queens.")
print(count_attacking_queens(cast("list[int]", solution.genotype)))

[10:16:37] Database file exists at                                                                       ]8;id=740908;file:///Users/jadenvanrijswijk/Downloads/ariel-dev/src/ariel/ec/a004.py\a004.py]8;;\:]8;id=518710;file:///Users/jadenvanrijswijk/Downloads/ariel-dev/src/ariel/ec/a004.py#98\98]8;;\
           /Users/jadenvanrijswijk/Downloads/ariel-dev/examples/__data__/database.db!                              
           Behaviour is set to: 'delete' --> ⚠️  Deleting file!                                                     

───────────────────────────────────────────────── EA Initialised ──────────────────────────────────────────────────

Output()

─────────────────────────────────────────────── EA Finished Running ───────────────────────────────────────────────

Best solution found:
[6, 0, 3, 5, 7, 2, 4, 1]
. . . . . . Q . 
Q . . . . . . . 
. . . Q . . . . 
. . . . . Q . . 
. . . . . . . Q 
. . Q . . . . . 
. . . . Q . . . 
. Q . . . . . . 
None
With 1 attacking queens.
1
